In [1]:
import cv2 as cv
import pybboxes as pbx
from ultralytics import YOLO
from ultralytics.yolo.utils.plotting import Annotator

WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")



In [2]:
model = YOLO('../best.pt')

In [3]:
class_names = open('../classes.txt').read().strip().splitlines()
class_names

['window', 'window-outside']

In [4]:
file_name = 'window_example2'
image = cv.imread(f'../{file_name}.jpg')
image = cv.resize(image, (600, 800), interpolation=cv.INTER_LINEAR)

height, width, _ = image.shape

bgr_image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
results = model.predict(bgr_image, conf=0.5, classes=0)
# annotator = Annotator(image)


0: 640x480 2 windows, 62.6ms
Speed: 4.7ms preprocess, 62.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 480)


In [5]:
for result in results:
    for box in result.boxes:
        class_id = int(box.cls)
        class_name = class_names[class_id]
        yolo_bbox = list(map(float, box.xywhn[0]))
        cv_bbox = pbx.convert_bbox(yolo_bbox, from_type="yolo", to_type="voc", image_size=(width, height))
            
        print(class_name)
        print(cv_bbox)
        cv.rectangle(image, cv_bbox[:2], cv_bbox[2:], (0, 255, 0), thickness=2)

cv.imshow(f'{file_name}_annotated', image)
cv.imwrite(f'{file_name}_annotated.jpg', image)
cv.waitKey()
cv.destroyAllWindows()

window
(97, 30, 495, 258)
window
(300, 251, 464, 672)
